## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBBSSB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
499
+-------------------+-------------------+-------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|            coeff_1|            coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|              res_2|              res_3|          res_mean|    station|   total_var|undefs|              vector|  year|
+-------------------+-------------------+-------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|-3975.9681279375172|-2226.5193552251353|-324.35763807368573|   2145.8|SSSBBSSB| 37.4403|-112.4819|       SNWD|0.3415303260823009|0.13149425317360844|0.12569642685188906|0.7615

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [24]:
#extract longitude and latitude for each station
feature='coeff_3'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


,station,year,coeff_3
0,USC00420086,1941.0,-324.357638
1,USC00420900,1994.0,-1597.854243
2,USC00420900,1999.0,380.884319
3,USC00420900,1992.0,-1710.563935
4,USC00421008,2005.0,1932.601131


In [25]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

station,USC00260099,USC00261358,USC00266252,USC00420086,USC00420738,USC00420757,USC00420788,USC00420849,USC00420900,USC00421006,...,USC00426534,USC00426601,USC00426686,USC00428456,USC00428847,USC00429136,USC00429514,USW00023159,USW00023177,USW00093129
year,,,,,,,,,,,,,,,,,,,,,
1917.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1929.0,NaN,NaN,NaN,-148.323810,-206.336528,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1930.0,NaN,NaN,NaN,-149.769979,NaN,NaN,NaN,NaN,NaN,NaN,...,-227.461576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1931.0,NaN,NaN,NaN,-338.026910,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1932.0,NaN,-109.210568,NaN,-97.164691,-1554.703310,NaN,-383.346178,NaN,NaN,NaN,...,-566.989783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1933.0,NaN,-284.820260,NaN,-517.982003,-954.822790,NaN,-206.914841,NaN,NaN,NaN,...,-257.399951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1934.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1935.0,NaN,NaN,NaN,101.699275,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936.0,NaN,NaN,NaN,-97.376419,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [26]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  852.415894433
RMS removing mean-by-station=  741.409501832
RMS removing mean-by-year   =  661.488079531


In [27]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 852.415894433
0 after removing mean by year    = 661.488079531
0 after removing mean by stations= 582.246074269
1 after removing mean by year    = 573.250613109
1 after removing mean by stations= 571.709631532
2 after removing mean by year    = 571.266052782
2 after removing mean by stations= 571.078806914
3 after removing mean by year    = 570.985270923
3 after removing mean by stations= 570.935454052
4 after removing mean by year    = 570.908129253
4 after removing mean by stations= 570.892874566


In [19]:
eigvals=STAT['SNWD']['eigval']; eigvals/=sum(eigvals); cumvar=cumsum(eigvals); cumvar=100*np.insert(cumvar,0,0)

NameError: name 'cumsum' is not defined